This Jupyter Notebook will be the main program from our project. When the project is finished, we will be able to use this Jupyter code to generate the presentation slides.

In [ ]:
import Scripts #Import all the scripts in the "Scripts" folder